In [1]:
import pandas as pd
import numpy as np
import os
import re
from fuzzysearch import find_near_matches, find_near_matches_in_file
os.chdir('/Users/jk8sd/Box Sync/Code for Cville')

In [2]:
names = pd.read_csv('BOP Interview Responses and Tracking - Sheet4(1).csv')
interviewees = names.query('Type == "Interviewee"')['Participant'].tolist()
interviewers = names.query('Type == "Interviewer"')['Participant'].tolist()

In [3]:
def cleantext(s, interviewer, interviewee, interviewee2=None):
    if interviewee2 is None:
        re_pass = '(' + re.escape(interviewee) + ' |' + re.escape(interviewer) + ')'
    else:
        re_pass = '(' + re.escape(interviewee) + ' |' + re.escape(interviewee2) + ' |' + re.escape(interviewer) + ')'
    s_split = re.split(re_pass, s)
    tags = [interviewer, interviewee, interviewee2]
    if s_split[0].strip() not in tags:
        s_split = s_split[1:]
    cleandf = pd.DataFrame(np.array(s_split).reshape((-1,2)))
    logic = cleandf.iloc[:,0] != interviewer
    cleandf = cleandf.loc[logic,]
    clean = ''.join(cleandf.iloc[:,1])
    return(clean)

In [4]:
def closematch(txt, g, verbose=False):
    if verbose:
        print(g)
    if txt.find(g) != -1:
        return(g)
    else:
        m = find_near_matches(g, txt, max_l_dist=10)
        if len(m) == 0:
            return(None)
        else: 
            return(m[np.argmin(np.array([j.dist for j in m]))].matched)

In [5]:
def parsefile(address, verbose=False):
    F = open(address)
    txt = F.read()
    
    int1 = np.array(interviewers)[np.array([i in txt for i in interviewers])]
    interviewer = int1[0]
    int2 = np.array(interviewees)[np.array([i in txt for i in interviewees])]
    interviewee = int2[0]
    interviewee2 = None
    if len(int2) > 1:
        interviewee2 = int2[1]
    
    txt = txt.lower()
    txt = txt.replace("i'm in a remote cricket", "running a remote brigade") #014_codeforbaltimore_2020-11-10_Jason-Anton.txt'
    txt = txt.replace("tuesday manager per day", "tools to manage your brigade") #013_codeforboulder_2020-11-03_Jacob-Venable.txt
    txt = txt.replace("any topics that's not a, that's not in the list", "are there any topics not in this list") #013_codeforboulder_2020-11-03_Jacob-Venable.txt
    txt = txt.replace("baudin corking", "building a core team")#013_codeforboulder_2020-11-03_Jacob-Venable.txt
    txt = txt.replace("processes that you find effective", "do you have any effective processes")#013_opencolumbus-ohio_2020-11-14_Vijay-Yadav_Mihir-Amin.txt
    txt = txt.replace("someone violates, how to deal with the why violation", "how to deal with code of conduct violations") #013_codefororlando_2020-10-31_Krystina-Ramos.txt
    txt = txt.replace("okay, and next one is fundraising", "drafting a strategic plan for a project okay, and next one is fundraising") #013_codefororlando_2020-10-31_Krystina-Ramos.txt
    q = ['hosting hack nights',
         'for example, hosting the national day of civic hacking',
         'cultivating government partnerships',
         'cultivating community partnerships',
         'hosting a workshop to help partners identify user needs',
         'practicing lean software development',
         'conducting user testing',
         'how to deal with code of conduct violations',
         'building a core team',
         'drafting a strategic plan for your brigade',
         'drafting a strategic plan for a project',
         'fundraising',
         'tools to manage your brigade',
         'developing a brand and media strategy',
         'onboarding to the national network',
         'guide for how to make open source projects',
         'running a remote brigade',
         'how to set and achieve',
         'connecting people with local government job opportunities',
         'workforce development',
         'are there any topics not in this list',
         'do you have any effective processes']

    q2 = [closematch(txt, g, verbose=verbose) for g in q]    
    
    diagnostic = pd.DataFrame({'script': q, 'transcript': q2, 'same': np.array(q)==np.array(q2)})
    
    q.insert(0, 'introduction')
    txtparse = []
    txtremain = txt

    for i in q2:
        if i is None:
            txtparse.append('')
        else:
            r = txtremain.split(i, maxsplit=1)
            txtparse.append(r[0])
            txtremain = r[1]

    txtparse.append(txtremain)
    
    interviewer = interviewer.lower()
    interviewee = interviewee.lower()
    if interviewee2 is not None:
        interviewee2 = interviewee2.lower()
    
    txtparse = [cleantext(i, interviewer, interviewee, interviewee2) for i in txtparse]

    parsedtext = pd.DataFrame([txtparse], columns=q) 
    return(diagnostic, parsedtext, interviewer, interviewee, interviewee2)


In [6]:
files = os.listdir('textfiles')
files.remove('.DS_Store')
brokenfiles = []
finaldata = pd.DataFrame()
c = 1

for f in files:
    print('file ' + str(c) + ' of ' + str(len(files)) + ": " + f)
    address = 'textfiles/' + f
    try: 
        diagnosis, parseddf, interviewer, interviewee, interviewee2 = parsefile(address)
        #parseddf['interviewer'] = interviewer
        parseddf['interviewee'] = interviewee
        parseddf['interviewee2'] = interviewee2
        parseddf['otteraifile'] = f
        finaldata = finaldata.append(parseddf)
    except:
        print('error in file ' + str(c) + ' of ' + str(len(files)) + ": " + f)
        brokenfiles = brokenfiles + [c]
    c = c + 1

file 1 of 37: 014_opennccollaborative_2020-11-21_Chris-Mathews.txt
file 2 of 37: 014_codeforchicago_2020-11-06_Ryan-Koch.txt
file 3 of 37: 014_opencleveland_2020-11-11_Will-Skora.txt
file 4 of 37: 003_codeforboston_2020-10-01_Harlan-Weber.txt
file 5 of 37: .DS_Store
error in file 5 of 37: .DS_Store
file 6 of 37: 013_opentwincities_2020-11-14_Eric-Ebbesen_Alison-Link.txt
file 7 of 37: 014_codeforbaltimore_2020-11-10_Jason-Anton.txt
file 8 of 37: 013_codeforsanjose_2020-10-19_Annie-Steenson.txt
file 9 of 37: 013_codeforboulder_2020-11-03_Jacob-Venable.txt
file 10 of 37: 022_opendatadelaware_2020-11-11_David-Ginzberg.txt
file 11 of 37: 018_codeformontana_2020-11-05_Smai-Fullerton.txt
file 12 of 37: 014_codeforsacramento_2020-11-05_Kevin-Fries.txt
file 13 of 37: 003_openoakland_2020-10-30_Thomas-Deckert.txt
file 14 of 37: 026_codeformilwaukee_2020-11-09_Andrew-Yaspan_part1.txt
file 15 of 37: 022_codeforphoenix_2020-11-23_Tom-Marthaler.txt
file 16 of 37: 002_codefordallas_2020-11-11_Hari-Ma

In [7]:
finaldata

,introduction,hosting hack nights,"for example, hosting the national day of civic hacking",cultivating government partnerships,cultivating community partnerships,hosting a workshop to help partners identify user needs,practicing lean software development,conducting user testing,how to deal with code of conduct violations,building a core team,...,guide for how to make open source projects,running a remote brigade,how to set and achieve,connecting people with local government job opportunities,workforce development,are there any topics not in this list,do you have any effective processes,interviewee,interviewee2,otteraifile
0,"0:43 \nit doesn't, i'm not here yet.\n\nunkn...",9:08 \nwe don't actually run hacknight this ...,"10:20 \num, we did not participate as a coll...","10:55 \ngive this fairly well. for instance,...","11:36 \ncan you be, again, i think we do tha...",12:15 \nwe do that once we did that once we ...,"12:48 \ni would say, again we do that fairly...","13:36 \nsee, we, we don't do it. we i wish w...",14:52 \nthe time. we will each individual br...,15:38 \nour core team is the is the leaders ...,...,19:58 \nwe actually a few years ago had some...,"20:33 \ni think we, we do that we do that we...",21:03 \nwe do that fairly well again well si...,"21:30 \noh, we have not really done a really...","22:14 \nwe do have a good, a fairly good net...","23:03 \nno, i think we just, i think it just...",24:04 \ni'd say yes. okay.\n\n 24:25 \ni wo...,chris (open nc collaborative),None,014_opennccollaborative_2020-11-21_Chris-Mathe...
0,16:31 \nhow you doing today i'm\n\n 16:37 \...,,21:48 \nyeah. should i just verbally answer ...,"23:20 \nso this is, i think, been a been a b...","24:29 \nso, so that one i think we do really...",25:46 \nan example there might be helpful. i...,26:20 \nso that one might be useful to see h...,27:03 \nso this one i think this is somethin...,28:23 \nexample could could be useful there....,"29:26 \ncore team, um,\n\ni say this i think...",...,"36:01 \nah, so i think that would be useful....","36:52 \nso, this one i actually think is som...","39:33 \nso that one,\n\nthat one, that one w...",40:54 \nlocal government job opportunities. ...,42:11 \nthat can also be useful. we don't ha...,"43:04 \nthat's a good question. oh,\n\none t...",47:11 \nall right. i think i probably for no...,ryan (code for chicago),None,014_codeforchicago_2020-11-06_Ryan-Koch.txt
0,6:55 \nokay sorry about sorry i couldn't hea...,,"12:36 \noh, sure. so. oh, sorry, sorry i've ...","19:41 \noh, oh yeah i said i guess it's thre...","22:54 \ni think we do this relatively well, ...","25:21 \ni would say actually five, um i thin...",27:03 \nactually we're kind of going we're k...,27:48 \ni would also go with three. we've do...,29:15 \ni'm sorry. could you repeat that one...,"32:12 \noh, um, that's something i've always...",...,"49:31 \nsir, with the scale again.\n\n 49:42...",53:03 \none through six. mm hmm. i'm in sort...,"54:27 \nactually six yeah, that's something ...",54:43 \ni think one. i think we actually do ...,55:05 \ncurrently 02 we don't do that much o...,55:38 \nprep six succession planning. okay. ...,"57:06 \nyeah, there's a couple small things,...",will (open cleveland),None,014_opencleveland_2020-11-11_Will-Skora.txt
0,1:43 \nyou want me to read it?\n\n 1:52 \ny...,"2:49 \nk? my question is like, like now eigh...",,"3:31 \num, so like, operationally running da...",4:52 \nsomewhere between two and three\n\nwe...,5:25 \ndepends on the person who's doing it....,5:34 \ni don't really know what you mean by ...,"5:47 \noh, for sure. and again, i think a lo...",6:37 \nalthough i think eight years we've ha...,,...,"10:04 \num, there wouldn't be there would be...",,,10:22 \nthe that would be very useful i thin...,11:26 \nlike you're talking about we're tryi...,"14:51 \nyou know, we are a big and old briga...","21:13 \na man yeah, probably. sure.\n\ni thi...",harlan (code for boston),None,003_codeforboston_2020-10-01_Harlan-Weber.txt


In [8]:
finaldata.to_csv('transcripttext.csv', index=False)

In [9]:
bf = list(files[i-1] for i in brokenfiles )
bf

['.DS_Store',
 '020_codefordurham_2020-11-24_Laura-Biediger.txt',
 '026_codeformilwaukee_Andrew-Yaspan_part0_introduction.txt',
 '027_openoakland_2020-11-16_Jess-Sand.txt']

In [23]:
address = 'textfiles/' + bf[1]

F = open(address)
txt = F.read()

if address=='textfiles/020_codefordurham_2020-11-24_Laura-Biediger.txt':
    txt = txt.replace("Unknown Speaker", "Laura (Code for Durham)")

int1 = np.array(interviewers)[np.array([i in txt for i in interviewers])]
interviewer = int1[0]
int2 = np.array(interviewees)[np.array([i in txt for i in interviewees])]
interviewee = int2[0]
interviewee2 = None
if len(int2) > 1:
    interviewee2 = int2[1]

txt = txt.lower()
txt = txt.replace("i'm in a remote cricket", "running a remote brigade") #014_codeforbaltimore_2020-11-10_Jason-Anton.txt'
txt = txt.replace("tuesday manager per day", "tools to manage your brigade") #013_codeforboulder_2020-11-03_Jacob-Venable.txt
txt = txt.replace("any topics that's not a, that's not in the list", "are there any topics not in this list") #013_codeforboulder_2020-11-03_Jacob-Venable.txt
txt = txt.replace("baudin corking", "building a core team")#013_codeforboulder_2020-11-03_Jacob-Venable.txt
txt = txt.replace("someone violates, how to deal with the why violation", "how to deal with code of conduct violations") #013_codefororlando_2020-10-31_Krystina-Ramos.txt
txt = txt.replace("processes that you find effective", "do you have any effective processes")#013_opencolumbus-ohio_2020-11-14_Vijay-Yadav_Mihir-Amin.txt
txt = txt.replace("drafting a strategic plan for your birthday", "drafting a strategic plan for your brigade") #013_codefororlando_2020-10-31_Krystina-Ramos.txt
txt = txt.replace("okay, and next one is fundraising", "drafting a strategic plan for a project okay, and next one is fundraising") #013_codefororlando_2020-10-31_Krystina-Ramos.txt

q = ['hosting hack nights',
     'for example, hosting the national day of civic hacking',
     'cultivating government partnerships',
     'cultivating community partnerships',
     'hosting a workshop to help partners identify user needs',
     'practicing lean software development',
     'conducting user testing',
     'how to deal with code of conduct violations',
     'building a core team',
     'drafting a strategic plan for your brigade',
     'drafting a strategic plan for a project',
     'fundraising',
     'tools to manage your brigade',
     'developing a brand and media strategy',
     'onboarding to the national network',
     'guide for how to make open source projects',
     'running a remote brigade',
     'how to set and achieve',
     'connecting people with local government job opportunities',
     'workforce development',
     'are there any topics not in this list',
     'do you have any effective processes']

q2 = [closematch(txt, g, verbose=True) for g in q]    

diagnostic = pd.DataFrame({'script': q, 'transcript': q2, 'same': np.array(q)==np.array(q2)})
diagnostic


hosting hack nights
for example, hosting the national day of civic hacking
cultivating government partnerships
cultivating community partnerships
hosting a workshop to help partners identify user needs
practicing lean software development
conducting user testing
how to deal with code of conduct violations
building a core team
drafting a strategic plan for your brigade
drafting a strategic plan for a project
fundraising
tools to manage your brigade
developing a brand and media strategy
onboarding to the national network
guide for how to make open source projects
running a remote brigade
how to set and achieve
connecting people with local government job opportunities
workforce development
are there any topics not in this list
do you have any effective processes


,script,transcript,same
0,hosting hack nights,hosting hack nights,True
1,"for example, hosting the national day of civic...",None,False
2,cultivating government partnerships,cultivating government partnerships,True
3,cultivating community partnerships,cultivating community partnerships,True
4,hosting a workshop to help partners identify u...,hosting a workshop to help partners identify u...,True
5,practicing lean software development,practicing lean software development,True
6,conducting user testing,conducting user testing,True
7,how to deal with code of conduct violations,how to deal with code of conduct violations,True
8,building a core team,building a core team,True
9,drafting a strategic plan for your brigade,defining a strategic plan for your brigade,False


In [22]:
if address=='textfiles/020_codefordurham_2020-11-24_Laura-Biediger.txt':
    print(txt)

Unknown Speaker  1:39  
Hello. Hey,

Claire Salinas  1:43  
are you just here like for support. I know I'm the person that, Oh, I'm so sorry. I had a different name under my thing, I apologize. Okay.

Unknown Speaker  1:54  
Okay.

Claire Salinas  1:56  
Um, is Patrick, and I think maybe,

Unknown Speaker  2:00  
um,

Claire Salinas  2:01  
yeah I don't know what that was about maybe that's my, okay, I'm sorry, that's the point. Okay, how are you doing good. How are you, 

pretty good yeah yeah i was like struggling this morning with dry eyes so I was trying I I'm like overdue for an eye appointment and my contacts, I probably not healthy to wear them anymore, so yeah and you know I'm like super good about taking them out each night, but, um, yeah, I don't know, I think it's just like you said staring at the screens it just gets old quick. 

No. 

Okay, um well again I'm so sorry for keeping you a couple minutes that was like, okay, cool, cool so all Um, okay. 

First day of my day off

In [ ]:
q.insert(0, 'introduction')
txtparse = []
txtremain = txt
for i in q2:
    print(i)
    if i is None:
        txtparse.append('')
    else:
        r = txtremain.split(i, maxsplit=1)
        txtparse.append(r[0])
        txtremain = r[1]

In [ ]:
diagnosis, parseddf, interviewer, interviewee, interviewee2 = parsefile(address)

In [ ]:
'strategic plan for a project' in txt

In [ ]:
txt

In [ ]:
address

In [ ]:
q2 = [closematch(txt, g, verbose=True) for g in q]    
diagnostic = pd.DataFrame({'script': q, 'transcript': q2, 'same': np.array(q)==np.array(q2)})
diagnostic

In [ ]:
m = find_near_matches("tools to manage", txt, max_l_dist=10)
m

In [ ]:
test = txt.split('drafting a strategic plan for your\n\njaco')
test[1]

In [ ]:
len(txtparse)

In [ ]:
txtparse.append(txtremain)

In [ ]:
interviewer = interviewer.lower()
interviewee = interviewee.lower()
if interviewee2 is not None:
    interviewee2 = interviewee2.lower()
txtparse = [cleantext(i, interviewer, interviewee, interviewee2) for i in txtparse]
#txtparse

In [ ]:
parsedtext = pd.DataFrame([txtparse], columns=q) 

In [ ]:
parsedtext

In [ ]:
parsedtext